# ARP（Address Resolution Protocol）

## 🎯 學習目標

- 了解 ARP 的基本作用
- 學會使用 ARP 指令檢視網路設備
- 認識 ARP 安全問題與基本防護

## 📋 課程內容

1. ARP 基礎概念
2. 常用 ARP 指令
3. ARP 安全認知

## 1. ARP 基礎概念

### 什麼是 ARP？
ARP 是一個網路協議，負責將 **IP 位址**轉換為 **MAC 位址**（網卡實體地址）。

### 為什麼需要 ARP？
- 電腦使用 **IP 位址** 在網路上找到目標（如 192.168.1.5）
- 網卡需要 **MAC 位址** 才能實際傳送資料（如 aa:bb:cc:dd:ee:ff）
- **ARP 的工作** 就是建立這兩者的對應關係

### 簡單運作流程
1. 電腦想連到某個 IP
2. 檢查 ARP 表格，看看有沒有對應的 MAC 位址
3. 如果沒有，就廣播詢問「誰是這個 IP？」
4. 目標設備回答「我是，我的 MAC 是 xxx」
5. 電腦記住這個對應關係，開始傳送資料

In [ ]:
# 檢視所有 ARP 項目
arp -a

# 只顯示數字格式（不解析主機名稱）
arp -an

# 檢查特定 IP 的 ARP 項目
arp 192.168.1.1

# 檢視預設閘道的 ARP 資訊
route -n get default | grep gateway

## 2. 如何讀懂 ARP 表格

### ARP 輸出範例
```
# macOS 輸出
router.local (192.168.1.1) at aa:bb:cc:dd:ee:ff on en0
phone.local (192.168.1.5) at 11:22:33:44:55:66 on en0

# Linux 輸出  
192.168.1.1 dev eth0 lladdr aa:bb:cc:dd:ee:ff REACHABLE
192.168.1.5 dev eth0 lladdr 11:22:33:44:55:66 STALE
```

### 重要欄位說明
- **IP 位址**：設備在網路上的地址
- **MAC 位址**：網卡的實體地址（格式：xx:xx:xx:xx:xx:xx）
- **介面**：你的網卡名稱（en0、eth0 等）
- **狀態**：REACHABLE=可連通，STALE=可能過時

## 3. ARP 安全問題（認知篇）

### 🚨 ARP 的安全風險

ARP 協議比較古老，設計時沒考慮安全性：
- **無驗證機制**：任何人都能假冒別人的身份
- **容易被欺騙**：電腦會相信收到的 ARP 回應

### 常見 ARP 攻擊

#### ARP 欺騙（ARP Spoofing）
- **攻擊方式**：駭客假冒路由器或其他設備的 MAC 位址
- **危險後果**：
  - 竊聽你的網路流量
  - 竊取帳號密碼
  - 修改你看到的網頁內容
  - 讓你無法正常上網

#### 如何發現攻擊？
- 網路突然變慢或斷線
- 同一個 IP 對應到不同的 MAC 位址
- 出現不認識的設備

### 🛡️ 基本防護方法

#### 個人用戶防護
1. **使用安全的 WiFi**：避免連接公共 WiFi
2. **定期檢查 ARP 表**：留意是否有異常設備
3. **使用 HTTPS 網站**：加密連線較安全
4. **保持系統更新**：修補安全漏洞

#### 簡單檢查方法
```bash
# 定期執行，記住正常的設備
arp -a

# 特別注意你的路由器 MAC 是否改變
ping 192.168.1.1  # 先ping路由器
arp -a | grep 192.168.1.1  # 檢查MAC位址
```

#### 發現問題時的應對
1. **立即中斷網路連線**
2. **清空 ARP 快取**：`sudo arp -a -d`（需要管理員權限）
3. **重新連接網路**
4. **檢查是否還有問題**

In [ ]:
# ARP 表格分析
arp -a | wc -l  # 計算設備數量

# 查找重複的 MAC 位址
arp -a | awk '{print $4}' | sort | uniq -d

# 顯示最近活躍的設備（頭 10 個）
arp -a | head -10

# 檢查本地網段的設備
arp -a | grep "192.168.1"

# 清空 ARP 快取（需要 sudo 權限）
# sudo arp -a -d

In [ ]:
# 進階 ARP 管理指令（需要管理員權限）

# 手動新增靜態 ARP 項目
# sudo arp -s 192.168.1.100 aa:bb:cc:dd:ee:ff

# 刪除特定 ARP 項目  
# sudo arp -d 192.168.1.100

# Linux 系統的 ARP 指令
# ip neigh show  # 顯示鄰居表
# ip neigh show dev eth0  # 顯示特定介面
# ip neigh flush all  # 清空所有 ARP 快取

In [ ]:
# 網路探測與 ARP 相關指令

# 快速 ping 本地網段觸發 ARP 更新
for i in {1..10}; do ping -c 1 -W 1 192.168.1.$i >/dev/null 2>&1 & done; wait

# 檢查網路介面資訊
ifconfig | grep -E "(inet |ether )"

# macOS 系統資訊
scutil --dns | grep nameserver

# 監控 ARP 流量（需要 sudo）
# sudo tcpdump arp